In [ ]:
"""
import os,sys
import glob
import numpy as np
import random
from PIL import Image
import pandas as pd
import nibabel as nib
from pathlib import Path
import imageio
import matplotlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.image as mpimg
from nibabel.viewers import OrthoSlicer3D
from pathlib import Path
from matplotlib import pylab as plt
plt.style.use("ggplot")
%matplotlib inline
from skimage.util import img_as_ubyte
import imageio
from sklearn.preprocessing import minmax_scale
import cv2
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
pd.set_option('display.max_colwidth', None)"""

In [ ]:
import os,sys
import glob
import numpy as np
import random
import pandas as pd
pd.set_option('display.max_colwidth', None)
from pathlib import Path
import imageio
import cv2
from matplotlib import pylab as plt
import matplotlib
from tensorflow.keras.models import Model, load_model, save_model

In [ ]:
#Corrected Dice Coefficient
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    

#IOU Coeffifficent
def iou(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [ ]:
#Loading Segmentation Model
model = load_model("C:/Brain Tumor Validation/models/BraTs2019mri_final_seg1.h5", custom_objects={'dice_coef': dice_coef, 'iou':iou, 'precision':precision, \
                                                                                                                      'sensitivity':sensitivity, 'specificity':specificity})

In [ ]:
import os

path ="C:/Brain Tumor Validation/Patients/"
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        filelist.append(os.path.join(root,file))

#print all the file names
for name in filelist:
    print(name)

In [ ]:
#creating image dataframe
df = pd.DataFrame({'Image_dir': filelist})

In [ ]:
#structuring imag path string
df["Image_dir"] = [path.replace("\\", "/") for path in df["Image_dir"]]

In [ ]:
df.head()

In [ ]:
#add a column for subfolder
folder =[]
for path in df["Image_dir"]:
    pth = Path(path)
    pth_part = pth.parts
    foldername = pth_part[3] + '_' + pth_part[4] + '_' + pth_part[5]
    folder.append(foldername)

df["subfolder"] = folder
df.head()   

In [ ]:
#add a column for image types
df["Image_type"] = ["scan" if img[-8:-4] == "scan" else "mask" for img in df["Image_dir"]]
df.sample(3)

In [ ]:
#seperating scans from images
df_scan = df.loc[df["Image_type"] == "scan"]
df_mask = df.loc[df["Image_type"] == "mask"]
(print(df_scan.shape, df_mask.shape))

In [ ]:
print(df_scan.columns, df_mask.columns)

In [ ]:
#renaming columns
df_scan.columns = ['scans_dir', 'scans_subfolder', 'scans_Image_type']
df_mask.columns = ['masks_dir', 'masks_subfolder', 'masks_Image_type']

In [ ]:
print(df_scan.columns, df_mask.columns)

In [ ]:
#aligning scans with masks
df_img = df_scan.merge(df_mask, how="left", left_on='scans_subfolder', right_on='masks_subfolder')
df_img

In [ ]:
df_img.columns

In [ ]:
df_img = df_img[['scans_dir', 'masks_dir']]
df_img.head()

## Segmentation Prediction

In [ ]:
for i in range(16):
    #idx = np.random.randint(0, len(df_img))
    
    imagePath = os.path.join(df_img['scans_dir'].iloc[i])
    maskPath = os.path.join(df_img['masks_dir'].iloc[i])
    print(imagePath)
    
    image = cv2.imread(imagePath)
    mask = cv2.imread(maskPath)
    
    img_mask = cv2.resize(mask ,(240, 240))
    mask = img_mask / 255
    
    img = cv2.resize(image ,(240, 240))
    img = img / 255
    img = img[np.newaxis, :, :, 1]
    pred=model.predict(img)

    plt.figure(figsize=(12,12))
    plt.subplot(1,4,1)
    plt.imshow(np.squeeze(img))
    plt.title('Original Image')
    plt.subplot(1,4,2)
    plt.imshow(mask)
    plt.title('Original Mask')
    plt.subplot(1,4,3)
    plt.imshow(np.squeeze(pred))
    plt.title('Prediction')
    plt.subplot(1,4,4)
    plt.imshow(np.squeeze(pred) > 0.5)
    plt.title('BinaryPrediction')
    plt.show()

## HighGrade / Low Grade Glioma Classification

In [ ]:
#model1 = load_model("C:/Brain Tumor Validation/models/BraTs2019mri_CLASS.h5")
model1 = load_model("C:\Voila\models\brats2019_imagenet_class.h5")

In [ ]:
#2 out of 44segments completly incorrectly labelled

In [ ]:
#df_img['TUMOR_CATEGORY'] = ["" for images in df_img["scans_dir"]]

In [ ]:
labels = {'HGG': 0, 'LGG': 1, 'NO_TUMOR': 2}

In [ ]:
df_img.head()

In [ ]:
prediction_list = []
for i in range(len(df_img)):
    idx = np.random.randint(0, len(df_img))
    
    imagePath = os.path.join(df_img['scans_dir'].iloc[idx])    
    
    image = cv2.imread(imagePath)    
    img = cv2.resize(image ,(256, 256))
    img = img / 255
    #img = img[np.newaxis, :, :, 1]
    img = np.expand_dims(img, axis=0)
    pred=model1.predict(img)
    mod_pred = np.argmax(pred, axis =1)   
    prediction_list.append(mod_pred[0])
    preds = ["HGG" if i == 0 else "LGG"  if i ==1 else "NT" for i in prediction_list]  
    

In [ ]:
df_img = df_img.copy()

In [ ]:
df_img['predictions'] = preds

In [ ]:
df_img

In [ ]:
#All images are High grade glioma's
accuracy = len(df_img.loc[df_img["predictions"]=="HGG"])/df_img.shape[0]
print("Total Accuracy for test images = ", accuracy * 100, ' %')

In [ ]:
df_img.to_csv("Results/image classification results.csv")

## Testing Older Model

In [ ]:
#Loading Segmentation Model
model3 = load_model("C:/Brain Tumor Validation/models/BraTs2020unet.h5", custom_objects={'dice_coef': dice_coef, 'iou':iou, 'precision':precision, \
                                                                                                                      'sensitivity':sensitivity, 'specificity':specificity})

In [ ]:

image = cv2.imread("C:/Brain Tumor Validation/Patients/patient_1/t1/ax +c/scan.png")
img = image[:, :, 1]
img = cv2.resize(img ,(240, 240))
#img = img / 255
img = np.expand_dims(img, axis=0)

In [ ]:

fig = plt.figure(figsize=(5,5))
pref_Tumor = model3.predict(img)
imgplot = plt.imshow(pref_Tumor)
plt.show()